In [14]:
import matplotlib.pyplot as plt
from arcworld.general_utils import plot_grid
from datasets import load_dataset

from matplotlib import colors
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import tqdm 

In [24]:
def grid_to_rgb_sized(grid, output_size=(128, 128)):
    """
    Convert a NumPy grid with integers (0-9) to an RGB image and resize to specified size.
    
    Args:
        grid (np.ndarray): 2D array with integer values between 0 and 9.
        output_size (tuple): Desired output size (width, height), default is (128, 128).
    
    Returns:
        np.ndarray: RGB image array of shape (height, width, 3) resized to output_size.
    """
    COLORMAP = colors.ListedColormap(
        ['#FFFFFF', '#0074D9', '#FF4136', '#2ECC40', '#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    
    # Validate input
    if not isinstance(grid, np.ndarray) or grid.ndim != 2 or not np.issubdtype(grid.dtype, np.integer) or grid.min() < 0 or grid.max() > 9:
        raise ValueError("Grid must be a 2D NumPy array with integers between 0 and 9")
    
    # Normalize grid values to [0, 1] for colormap
    norm = colors.Normalize(vmin=0, vmax=9)
    # Apply colormap to get RGB values
    rgb = COLORMAP(norm(grid))
    # Convert to uint8 RGB image (remove alpha channel)
    rgb_uint8 = (rgb[:, :, :3] * 255).astype(np.uint8)
    
    # Resize using PIL
    img = Image.fromarray(rgb_uint8)
    img_resized = img.resize(output_size, resample=Image.Resampling.NEAREST)
    
    return np.array(img_resized)

# Demo
dataset = load_dataset("yassinetb/COGITAO", data_files={"data": "CompGen/exp_setting_1/experiment_1/test.parquet"})
img = grid_to_rgb_sized(np.array(dataset["data"][99]["input"]))
img_pil = Image.fromarray(img)
img_pil.save("test_image_high.jpg", format="JPEG", quality=100, optimize=True, subsampling=0, progressive=True)  # Very high quality, larger file


In [25]:
splits = ["train", "val", "test", "val_ood", "test_ood"]
exp_settings = [1,2,3]
experiments = [1,2,3,4,5]

# CompGen 

for exp_s in exp_settings:
    print(f"Processing Experiment Setting {exp_s}")
    for exp in experiments:
        print(f"  Processing Experiment {exp}")
        for split in splits:

            data_path = f"CompGen/exp_setting_{exp_s}/experiment_{exp}/{split}.parquet"
            saving_path = f"CompGen_Naturalistic/exp_setting_{exp_s}/experiment_{exp}/{split}/"

            data_dictionary = {}

            # Create directories if they don't exist
            import os
            os.makedirs(saving_path, exist_ok=True)
            try:
                dataset = load_dataset("yassinetb/COGITAO", data_files={"data": data_path})
                print(f"Loaded {data_path} successfully.")
            except Exception as e:
                print(f"Failed to load {data_path}: {e}")

            for i in tqdm.tqdm(range(len(dataset["data"]))):
                input = dataset["data"][i]["input"]
                output = dataset["data"][i]["output"]
                transformation = dataset["data"][i]["transformation_suite"]
                task_key = dataset["data"][i]["task_key"]
                data_dictionary[task_key] = {
                    "transformation": transformation
                }

                # Convert input and output grids to RGB images
                input_img = grid_to_rgb_sized(np.array(input))
                output_img = grid_to_rgb_sized(np.array(output))

                # Save images with high quality in the corresponding directories
                input_img_pil = Image.fromarray(input_img)
                output_img_pil = Image.fromarray(output_img)

                input_img_pil.save(f"{saving_path}{task_key}_input.png", format="JPEG", quality=100, optimize=True, subsampling=0, progressive=True)
                output_img_pil.save(f"{saving_path}{task_key}_output.png", format="JPEG", quality=100, optimize=True, subsampling=0, progressive=True)

            # Save the data dictionary as a NumPy file
            np.save(f"{saving_path}data_dictionary.npy", data_dictionary)

Processing Experiment Setting 1
  Processing Experiment 1
Loaded CompGen/exp_setting_1/experiment_1/train.parquet successfully.


100%|██████████| 100000/100000 [02:32<00:00, 656.89it/s]


Loaded CompGen/exp_setting_1/experiment_1/val.parquet successfully.


100%|██████████| 1000/1000 [00:01<00:00, 696.96it/s]


Loaded CompGen/exp_setting_1/experiment_1/test.parquet successfully.


...

100%|██████████| 1000/1000 [00:01<00:00, 623.30it/s]
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Generating data split: 1000 examples [00:00, 145393.23 examples/s]


Loaded CompGen/exp_setting_3/experiment_5/val_ood.parquet successfully.


100%|██████████| 1000/1000 [00:01<00:00, 612.25it/s]
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Generating data split: 1000 examples [00:00, 142044.97 examples/s]


Loaded CompGen/exp_setting_3/experiment_5/test_ood.parquet successfully.


100%|██████████| 1000/1000 [00:01<00:00, 606.49it/s]
